In [180]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

import ribasim

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)



In [181]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
results_dir = Path(base_dir, "results")
network_name = "zutphen_tki_netwerk"

In [183]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden\\wrij_afwateringseenheden_selectie_Zutphen.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['GFEIDENT', 'geometry']]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "fiona\ogrext.pyx", line 136, in fiona.ogrext.gdal_open_vector
  File "fiona\_err.pyx", line 291, in fiona._err.exc_wrap_pointer
fiona._err.CPLE_OpenFailedError: ..\..\ribasim_lumping_data\afw_eenheden\wrij_afwateringseenheden_selectie_Zutphen.shp: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\envs\ribasim\Lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\NLTAND\AppData\Local\Temp\ipykernel_8232\1770911492.py", line 3, in <module>
    areas_gdf = gpd.read_file(areas_file_path)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\Anaconda3\envs\ribasim\Lib\site-packages\geopandas\io\file.py", line 259, in _read_file
    return _read_file_fiona(
           ^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\Anaconda3\envs\ribasim

In [184]:
# Create networkanalysis
network = RibasimLumpingNetwork(
    name=network_name, 
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)

In [185]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj_data"],
    simulation_output_dir="FlowFM\\output",
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
);

 - Simulation set (winter): tki_zuthpen_berkel_basis.dsproj_data | Timestamps: 9 | his.nc and map.nc


In [186]:
simulation_name = 'tki_zuthpen_berkel_basis.dsproj_data'
file_bc = f'{base_dir}\\d-hydro\\{simulation_name}\\FlowFM\\input\\FlowFM_boundaryconditions1d.bc'

In [187]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data(file_bc)
# Export to geopackage
# network.export_to_geopackage()

D-HYDRO-network analysed:
 - nodes and waterlevels
 - edges and discharges
 - stations / pumps / weirs / orifices / bridges / culverts / uniweirs / confluences / bifurcations / boundaries


In [188]:
# Define locations where the network should be split into Ribasim basins:

network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kst_ST80830001', 'kst_ST80810015', # onderdoorlaten bij verdeelpunt De Berkel (Zutphen)
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151' # duikers voor wijk Zuidwijken
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
    ], 
    node_ids_to_include=[
        # 419, # voorbeeld splitsing
        # 1455, # extra punt rondom verdeelpunt De Berkel
    ],
    node_ids_to_exclude=[],
);

67 split locations
 - pump: 4
 - weir: 46
 - uniweir: 9
 - orifice: 2
 - culvert: 6


In [189]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

Create basins using split nodes:
 - create network graph from nodes/edges
 - split network graph at split locations
 - define numbers Ribasim-Basins and join edges/nodes
 - check whether each split location results in a split
 - create final locations Ribasim-Basins
 - define for each Ribasim-Basin the related basin area
 - create Ribasim-Edges between Boundary and Basin
 - create Ribasim-Edges between Basins and split locations
Exporting to geopackage:
 - areas, nodes, edges, stations, pumps, weirs, orifices, bridges, culverts, uniweirs, basin_areas, split_nodes, basins, basin_connections, boundaries, boundary_basin_connections, 
 - not available: 
Export location: ..\..\ribasim_lumping_data\results\zutphen_tki_netwerk\ribasim_network.qgz


Export to ribasim

nodes

In [190]:
# set id's to node. Start with basins, then boundaries and then splitnodes. start with id 1
basins_gdf =network.basins_gdf.copy()
basins_gdf['node_id'] = basins_gdf['basin'] + 1
basins_gdf['type'] = 'Basin'

boundaries_gdf = network.boundaries_gdf.copy()
boundaries_gdf['node_id'] = boundaries_gdf['boundary_id'] + len(network.basins_gdf) +1
boundaries_gdf['type'] = 'LevelBoundary'

splitnodes_gdf = network.split_nodes.copy()
splitnodes_gdf.insert(0, 'splitnode_id', range(len(splitnodes_gdf)))
splitnodes_gdf['node_id'] = splitnodes_gdf['splitnode_id'] + len(network.basins_gdf) + len(network.boundaries_gdf) +1
splitnodes_gdf['type'] = 'TabulatedRatingCurve' 
splitnodetypes = {
    'weir': 'TabulatedRatingCurve', 
    'uniweir': 'TabulatedRatingCurve' ,
    'pump': 'Pump', 
    'weir': 'TabulatedRatingCurve', 
    'culvert':'ManningResistance', 
    'manual': 'ManningResistance',
    'orifice' : 'TabulatedRatingCurve'
}
for nodetype in splitnodetypes:
    splitnodes_gdf.loc[splitnodes_gdf['split_type']==nodetype, 'type'] = splitnodetypes[nodetype]

# concat nodes
ribasim_node_gdf = pd.concat([basins_gdf, boundaries_gdf,splitnodes_gdf]).set_crs(basins_gdf.crs)
ribasim_node_gdf = ribasim_node_gdf.set_index('node_id')
ribasim_node_gdf = ribasim_node_gdf[['geometry', 'type']]
node = ribasim.Node(static=ribasim_node_gdf)

edges

In [195]:
basin_connections_gdf = network.basin_connections_gdf[['mesh1d_node_id', 'basin_in','basin_out','geometry']]

# merge to find splitnode id
basin_connections_gdf = basin_connections_gdf.merge(splitnodes_gdf[['splitnode_id','mesh1d_node_id', 'node_id']], left_on='mesh1d_node_id', right_on='mesh1d_node_id')

# split connections in the connections upstream and downstream of splitnode
# add node ID's 
basin_connections_gdf_us = basin_connections_gdf.copy()
basin_connections_gdf_us['geometry'] = basin_connections_gdf_us.geometry.apply(lambda x: LineString([x.coords[0], x.coords[1]]))
basin_connections_gdf_us['from_node_id'] = basin_connections_gdf_us['basin_out'] +1
basin_connections_gdf_us['to_node_id'] = basin_connections_gdf_us['node_id']

basin_connections_gdf_ds = basin_connections_gdf.copy()
basin_connections_gdf_ds['geometry'] = basin_connections_gdf.geometry.apply(lambda x: LineString([x.coords[1], x.coords[2]]))
basin_connections_gdf_ds['from_node_id'] = basin_connections_gdf_ds['node_id']
basin_connections_gdf_ds['to_node_id'] = basin_connections_gdf_ds['basin_in'] +1


# boundary basin connections - add node ID's
boundary_basin_connections = network.boundary_basin_connections_gdf[['boundary_id', 'basin','geometry','boundary_location']].copy()

boundary_basin_connections_us = boundary_basin_connections.loc[boundary_basin_connections['boundary_location'] == 'upstream'].copy()
boundary_basin_connections_us['from_node_id'] = boundary_basin_connections_us['boundary_id']  + len(network.basins_gdf) +1
boundary_basin_connections_us['to_node_id'] = boundary_basin_connections_us['basin'] +1

boundary_basin_connections_ds = boundary_basin_connections.loc[boundary_basin_connections['boundary_location'] == 'downstream'].copy()
boundary_basin_connections_ds['from_node_id'] = boundary_basin_connections_ds['basin'] +1
boundary_basin_connections_ds['to_node_id'] = boundary_basin_connections_ds['boundary_id'] + len(network.basins_gdf) +1

# Setup the edges:
ribasim_edges = pd.concat([basin_connections_gdf_ds, basin_connections_gdf_us,boundary_basin_connections_us, boundary_basin_connections_ds]) 
ribasim_edges = ribasim_edges[['from_node_id','to_node_id','geometry']].reset_index()
ribasim_edges['from_node_id'].astype(int)

edge = ribasim.Edge(static=ribasim_edges)


basin

In [213]:
dummyvalue = 5.5

# basin
profile_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Basin'].index.values.tolist()
    }
)
profile_data['storage'] = dummyvalue
profile_data['area'] = dummyvalue
profile_data['level'] = dummyvalue

static_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Basin'].index.values.tolist()
    }
)
static_data['drainage'] = dummyvalue
static_data['potential_evaporation'] = dummyvalue
static_data['infiltration'] = dummyvalue
static_data['precipitation'] = dummyvalue
static_data['urban_runoff'] = dummyvalue

basin = ribasim.Basin(profile=profile_data, static=static_data)

rating curve

In [214]:
# tabulated_rating_curve
static_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='TabulatedRatingCurve'].index
    }
)
static_data['level'] = dummyvalue
static_data['discharge'] = dummyvalue

tabulated_rating_curve = ribasim.TabulatedRatingCurve(static=static_data)

In [215]:
# manning_resistance
static_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='ManningResistance'].index
    }
)
static_data['length'] = dummyvalue
static_data['manning_n'] = dummyvalue
static_data['profile_width'] = dummyvalue
static_data['profile_slope'] = dummyvalue

manning_resistance = ribasim.ManningResistance(static= static_data)

boundary

In [216]:
# level_boundary
static_boundary = boundaries_gdf[['node_id']].copy()
static_boundary['level'] = dummyvalue

level_boundary = ribasim.LevelBoundary(static=static_boundary)

pump

In [217]:
# pump
static_pump = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Pump'].index.values.tolist()
    }
)
static_pump['flow_rate'] = 0.0

pump = ribasim.Pump(static=static_pump)

Export everything to geopackage

In [219]:
# model
model = ribasim.Model(
    modelname="ribasim_model",
    node=node,
    edge=edge,
    basin=basin,
    level_boundary=level_boundary,
    pump=pump,
    tabulated_rating_curve=tabulated_rating_curve,
    manning_resistance=manning_resistance, 
    starttime="2020-01-01 00:00:00",
    endtime="2021-01-01 00:00:00",
)

# Write the model to a TOML and GeoPackage:
# model.write(f"{results_dir}/{network.name}")